## Descrition

- Course: STAT638, 2022Fall
- Deadline: 2022/10/29, 12:01 pm
> Read Chapter 10 in the Hoff book


In [1]:
using Statistics

## Problem HW10-1

> Assume we have $4$ observations, $(-1,0,1,10)$, where the last observation can be thought of as an outlier. Assume that conditional on an unknown parameter $\theta$, the data area i.i.d. from some population distribution. Assume a standard normal prior for $\theta$.

### (1)

> First, assume that the population distribution is a normal distribution with mean $\theta$ and variance $1$. Draw samples from the posterior of $\theta$ using a Metropolis algorithm and also derive the exact posterior in closed form.


In [2]:
data = [-1., 0., 1., 10.]

4-element Vector{Float64}:
 -1.0
  0.0
  1.0
 10.0

**Exact**


In [3]:
mean(data)

2.5

In [4]:
1 / var(data)

0.03896103896103896

$$\theta \sim Normal(\mu_0=0, \tau^{2}_{0}=1)$$
$$Y \sim Normal(\theta, \sigma^2 = 1)$$

\begin{align}
  P(\theta | y, \sigma^2)%
  &\propto \exp\{-\frac{1}{2\tau^{2}_{0}}(\theta - \mu_0)^2\}\exp\{-\frac{1}{2\sigma^2}\sum(y_i - \theta)^2\}\\
  &= \exp\{-\frac{1}{2}\theta^2\}\exp\{-\frac{1}{2}\sum(y_i - \theta)^2\}\\
\end{align}

### (2)

> Now assume the population distribution is a Cauchy distribution with location parameter $\theta$ and scale $1$. (This is equivalent to a nonstandardized t distribution with one degree of freedom and location parameter $\theta$.) Draw samples from the posterior using the Metropolis algorithm.

### (3)

> Plot the exact posterior density from part $1$, together with kernel density estimates from the two Metropolis samplers. Describe how the outlier has affected the posteriors.